# EKS CSI FSX Lustre Setup

Amazon FSx for Lustre is a high-performance file system optimized for deep learning workloads. FSx provides POSIX-compliant file system access to S3 for multiple readers and writers simultaneously.
  
The Amazon FSx for Lustre Container Storage Interface (CSI) driver provides a CSI interface that allows Amazon EKS clusters to manage the lifecycle of Amazon FSx for Lustre file systems.  

https://docs.aws.amazon.com/eks/latest/userguide/fsx-csi.html

# 1. Install the FSx CSI Driver for Kubernetes

In [2]:
!eksctl utils associate-iam-oidc-provider \
    --region us-west-2 \
    --cluster cluster \
    --approve

[ℹ]  eksctl version 0.27.0
[ℹ]  using region us-west-2
[!]  retryable error (RequestError: send request failed
caused by: Put "http://169.254.169.254/latest/api/token": context deadline exceeded (Client.Timeout exceeded while awaiting headers)) from ec2metadata/GetToken - will retry after delay of 31.122762ms
[!]  retryable error (RequestError: send request failed
caused by: Put "http://169.254.169.254/latest/api/token": context deadline exceeded (Client.Timeout exceeded while awaiting headers)) from ec2metadata/GetToken - will retry after delay of 60.368558ms
[!]  retryable error (RequestError: send request failed
caused by: Put "http://169.254.169.254/latest/api/token": context deadline exceeded (Client.Timeout exceeded while awaiting headers)) from ec2metadata/GetToken - will retry after delay of 190.17724ms
[!]  retryable error (RequestError: send request failed
caused by: Put "http://169.254.169.254/latest/api/token": context deadline exceeded (Client.Timeout exceeded while awaiti

In [14]:
!pygmentize eks-csi-fsx/fsx-csi-driver.json

{
    "Version":"2012-10-17",
    "Statement":[
        {
            "Effect":"Allow",
            "Action":[
                "iam:CreateServiceLinkedRole",
                "iam:AttachRolePolicy",
                "iam:PutRolePolicy"
            ],
            "Resource":"arn:aws:iam::*:role/aws-service-role/s3.data-source.lustre.fsx.amazonaws.com/*"
        },
        {
            "Action":"iam:CreateServiceLinkedRole",
            "Effect":"Allow",
            "Resource":"*",
            "Condition":{
                "StringLike":{
                    "iam:AWSServiceName":[
                        "fsx.amazonaws.com"
                    ]
                }
            }
        },
        {
            "Effect":"Allow",
            "Action":[
                "s3:ListBucket",
                "fsx:CreateFileSystem",
                "fsx:DeleteFileSystem",
                "fsx:DescribeFileSystems"
            ],
            "Resource":[
                "*"
            ]
        }
    ]

In [16]:
!aws iam create-policy \
    --policy-name Amazon_FSx_Lustre_CSI_Driver \
    --policy-document file://eks-csi-fsx/fsx-csi-driver.json

{
    "Policy": {
        "PolicyName": "Amazon_FSx_Lustre_CSI_Driver",
        "PolicyId": "ANPATLVNRE7W3KS26ZFGO",
        "Arn": "arn:aws:iam::231218423789:policy/Amazon_FSx_Lustre_CSI_Driver",
        "Path": "/",
        "DefaultVersionId": "v1",
        "AttachmentCount": 0,
        "PermissionsBoundaryUsageCount": 0,
        "IsAttachable": true,
        "CreateDate": "2020-10-29T14:48:18Z",
        "UpdateDate": "2020-10-29T14:48:18Z"
    }
}


In [17]:
policy_arn= 'arn:aws:iam::231218423789:policy/Amazon_FSx_Lustre_CSI_Driver'

In [18]:
!eksctl create iamserviceaccount \
    --region us-west-2 \
    --name fsx-csi-controller-sa \
    --namespace kube-system \
    --cluster cluster \
    --attach-policy-arn $policy_arn \
    --approve

[ℹ]  eksctl version 0.27.0
[ℹ]  using region us-west-2
[!]  retryable error (RequestError: send request failed
caused by: Put "http://169.254.169.254/latest/api/token": context deadline exceeded (Client.Timeout exceeded while awaiting headers)) from ec2metadata/GetToken - will retry after delay of 44.729366ms
[!]  retryable error (RequestError: send request failed
caused by: Put "http://169.254.169.254/latest/api/token": context deadline exceeded (Client.Timeout exceeded while awaiting headers)) from ec2metadata/GetToken - will retry after delay of 93.128318ms
[!]  retryable error (RequestError: send request failed
caused by: Put "http://169.254.169.254/latest/api/token": context deadline exceeded (Client.Timeout exceeded while awaiting headers)) from ec2metadata/GetToken - will retry after delay of 124.550252ms
[!]  retryable error (RequestError: send request failed
caused by: Put "http://169.254.169.254/latest/api/token": context deadline exceeded (Client.Timeout exceeded while await

In [19]:
cf_stack_name = 'eksctl-cluster-addon-iamserviceaccount-kube-system-fsx-csi-controller-sa'

In [20]:
role_name = 'eksctl-cluster-addon-iamserviceaccount-kube-Role1-7L9YH5501XP7'

In [21]:
role_arn = 'arn:aws:iam::231218423789:role/eksctl-cluster-addon-iamserviceaccount-kube-Role1-7L9YH5501XP7'

## Deploy CSI Driver

In [22]:
!kubectl apply -k "github.com/kubernetes-sigs/aws-fsx-csi-driver/deploy/kubernetes/overlays/stable/?ref=master"

serviceaccount/fsx-csi-controller-sa configured
clusterrole.rbac.authorization.k8s.io/fsx-csi-external-provisioner-role created
clusterrolebinding.rbac.authorization.k8s.io/fsx-csi-external-provisioner-binding created
deployment.apps/fsx-csi-controller created
daemonset.apps/fsx-csi-node created
csidriver.storage.k8s.io/fsx.csi.aws.com created


In [215]:
!kubectl describe csidriver.storage.k8s.io/fsx.csi.aws.com

Name:         fsx.csi.aws.com
Namespace:    
Labels:       <none>
Annotations:  kubectl.kubernetes.io/last-applied-configuration:
                {"apiVersion":"storage.k8s.io/v1beta1","kind":"CSIDriver","metadata":{"annotations":{},"name":"fsx.csi.aws.com"},"spec":{"attachRequired":...
API Version:  storage.k8s.io/v1beta1
Kind:         CSIDriver
Metadata:
  Creation Timestamp:  2020-10-29T15:04:13Z
  Resource Version:    28721012
  Self Link:           /apis/storage.k8s.io/v1beta1/csidrivers/fsx.csi.aws.com
  UID:                 c7cc13b8-d931-47a4-8d6c-869d89085b03
Spec:
  Attach Required:    false
  Pod Info On Mount:  false
  Volume Lifecycle Modes:
    Persistent
Events:  <none>


In [23]:
!kubectl annotate serviceaccount -n kube-system fsx-csi-controller-sa \
    eks.amazonaws.com/role-arn=$role_arn --overwrite=true

serviceaccount/fsx-csi-controller-sa annotated


# Test the CSI Driver

In [26]:
!aws s3 mb s3://fsx-csi-231218423789

make_bucket: fsx-csi-231218423789


In [27]:
!echo test-file >> testfile

In [28]:
!aws s3 cp testfile s3://fsx-csi-231218423789/export/testfile

upload: ./testfile to s3://fsx-csi-231218423789/export/testfile   


# Download Storage Class Manifest

In [29]:
#!curl -o storageclass.yaml https://raw.githubusercontent.com/kubernetes-sigs/aws-fsx-csi-driver/master/examples/kubernetes/dynamic_provisioning_s3/specs/storageclass.yaml

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   336  100   336    0     0   1647      0 --:--:-- --:--:-- --:--:--  1639


In [197]:
!pygmentize eks-csi-fsx/storageclass.yaml

kind: StorageClass
apiVersion: storage.k8s.io/v1
metadata:
  name: fsx-sc
provisioner: fsx.csi.aws.com
parameters:
  subnetId: subnet-0394df757ad94fdf0 
  securityGroupIds: sg-08de51a009f6896c4
  autoImportPolicy: NEW
  s3ImportPath: s3://fsx-csi-231218423789
  s3ExportPath: s3://fsx-csi-231218423789
  deploymentType: SCRATCH_2


In [33]:
%%bash

source ~/.bash_profile

#### Get VPC ID
export VPC_ID=$(aws ec2 describe-vpcs --filters "Name=tag:Name,Values=eksctl-${AWS_CLUSTER_NAME}-cluster/VPC" --query "Vpcs[0].VpcId" --output text)
echo "export VPC_ID=${VPC_ID}" | tee -a ~/.bash_profile

#### Get Subnet ID
export SUBNET_ID=$(aws ec2 describe-subnets --filters "Name=vpc-id,Values=${VPC_ID}" --query "Subnets[0].SubnetId" --output text)
echo "export SUBNET_ID=${SUBNET_ID}" | tee -a ~/.bash_profile

aws ec2 create-security-group --group-name eks-fsx-security-group4 --vpc-id ${VPC_ID} --description "FSx for Lustre Security Group" --query "GroupId" --output text


export VPC_ID=vpc-05cd6f5a363b289c2
export SUBNET_ID=subnet-0394df757ad94fdf0
sg-08de51a009f6896c4
None


In [36]:
security_group='sg-08de51a009f6896c4'

In [51]:
!aws ec2 authorize-security-group-ingress --group-id $security_group --protocol tcp --port 988 --cidr 192.168.0.0/16

## _Update storageclass.yaml_
https://docs.aws.amazon.com/fsx/latest/LustreGuide/getting-started-step1.html
  
https://github.com/kubernetes-sigs/aws-fsx-csi-driver/blob/master/examples/kubernetes/dynamic_provisioning_s3/README.md

In [198]:
!pygmentize eks-csi-fsx/storageclass.yaml

kind: StorageClass
apiVersion: storage.k8s.io/v1
metadata:
  name: fsx-sc
provisioner: fsx.csi.aws.com
parameters:
  subnetId: subnet-0394df757ad94fdf0 
  securityGroupIds: sg-08de51a009f6896c4
  autoImportPolicy: NEW
  s3ImportPath: s3://fsx-csi-231218423789
  s3ExportPath: s3://fsx-csi-231218423789
  deploymentType: SCRATCH_2


In [190]:
!kubectl delete -f eks-csi-fsx/storageclass.yaml

storageclass.storage.k8s.io "fsx-sc" deleted


In [199]:
!kubectl apply -f eks-csi-fsx/storageclass.yaml

storageclass.storage.k8s.io/fsx-sc created


In [201]:
!kubectl get sc

NAME                        PROVISIONER                                                RECLAIMPOLICY   VOLUMEBINDINGMODE      ALLOWVOLUMEEXPANSION   AGE
fsx-sc                      fsx.csi.aws.com                                            Delete          Immediate              false                  56s
local-hostpath              openebs.io/local                                           Delete          WaitForFirstConsumer   false                  9h
openebs-device              openebs.io/local                                           Delete          WaitForFirstConsumer   false                  9h
openebs-hostpath            openebs.io/local                                           Delete          WaitForFirstConsumer   false                  9h
openebs-jiva-default        openebs.io/provisioner-iscsi                               Delete          Immediate              false                  9h
openebs-snapshot-promoter   volumesnapshot.external-storage.k8s.io/snapshot-promoter  

In [38]:
#!curl -o claim.yaml https://raw.githubusercontent.com/kubernetes-sigs/aws-fsx-csi-driver/master/examples/kubernetes/dynamic_provisioning_s3/specs/claim.yaml

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   188  100   188    0     0    949      0 --:--:-- --:--:-- --:--:--   944


In [202]:
!pygmentize eks-csi-fsx/claim.yaml

apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: fsx-claim
spec:
  accessModes:
    - ReadWriteMany
  storageClassName: fsx-sc
  resources:
    requests:
      storage: 1200Gi


In [187]:
!kubectl delete -f eks-csi-fsx/claim.yaml

persistentvolumeclaim "fsx-claim" deleted


In [203]:
!kubectl apply -f eks-csi-fsx/claim.yaml

persistentvolumeclaim/fsx-claim created


In [206]:
!kubectl get pvc fsx-claim

NAME        STATUS   VOLUME                                     CAPACITY   ACCESS MODES   STORAGECLASS   AGE
fsx-claim   Bound    pvc-61624c47-dd5b-4214-9455-c55560294d53   1200Gi     RWX            fsx-sc         8m40s


In [71]:
#!kubectl describe pvc fsx-claim

## _Wait for status == Bound_

# Deploy Pod

https://github.com/kubernetes-sigs/aws-fsx-csi-driver/tree/master/examples/kubernetes/dynamic_provisioning_s3

In [111]:
#!pygmentize bert-csi-fsx.yaml

In [226]:
!kubectl create -f bert-csi-fsx.yaml

pod/bert-csi-fsx created


In [224]:
!kubectl delete -f bert-csi-fsx.yaml

pod "bert-csi-fsx" deleted


In [227]:
!kubectl get pod bert-csi-fsx

NAME           READY   STATUS    RESTARTS   AGE
bert-csi-fsx   1/1     Running   0          5s


In [222]:
!kubectl describe pod bert-csi-fsx

Name:         bert-csi-fsx
Namespace:    kubeflow
Priority:     0
Node:         ip-192-168-67-206.us-west-2.compute.internal/192.168.67.206
Start Time:   Thu, 29 Oct 2020 20:45:25 +0000
Labels:       <none>
Annotations:  kubernetes.io/psp: eks.privileged
Status:       Running
IP:           192.168.64.46
Containers:
  bert:
    Container ID:  docker://637ab2a8bca91218962773895dac6bd904dc018702b713c94dd2071384ddec6a
    Image:         763104351884.dkr.ecr.us-west-2.amazonaws.com/tensorflow-training:2.1.0-cpu-py36-ubuntu18.04
    Image ID:      docker-pullable://763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training@sha256:4911ac31a130c68a2f92b72dd81d22bd02b542cc549c5652f22c1f24e702eaf5
    Port:          <none>
    Host Port:     <none>
    Command:
      python
      /opt/ml/code/train.py
      --train_steps_per_epoch=1
      --epochs=1
      --learning_rate=0.00001
      --epsilon=0.00000001
      --train_batch_size=36
      --validation_batch_size=18
      --test_batch_size=

In [228]:
!kubectl logs -f bert-csi-fsx

  Created wheel for sacremoses: filename=sacremoses-0.0.43-py3-none-any.whl size=893259 sha256=f56cc45056714486837254b38560390e93016b909fc3aa2f778e503daabe5845
  Stored in directory: /root/.cache/pip/wheels/49/25/98/cdea9c79b2d9a22ccc59540b1784b67f06b633378e97f58da2
Successfully built sacremoses
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
  Attempting uninstall: smdebug
    Found existing installation: smdebug 0.7.2
    Uninstalling smdebug-0.7.2:
      Successfully uninstalled smdebug-0.7.2
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.22
    Uninstalling scikit-learn-0.22:
      Successfully uninstalled scikit-learn-0.22
You should consider upgrading via the '/usr/local/bin/python -m pip instal

In [127]:
!kubectl get sc

NAME                        PROVISIONER                                                RECLAIMPOLICY   VOLUMEBINDINGMODE      ALLOWVOLUMEEXPANSION   AGE
ebs-sc (default)            ebs.csi.aws.com                                            Delete          WaitForFirstConsumer   false                  6h36m
fsx-sc                      fsx.csi.aws.com                                            Delete          Immediate              false                  114m
local-hostpath              openebs.io/local                                           Delete          WaitForFirstConsumer   false                  7h59m
openebs-device              openebs.io/local                                           Delete          WaitForFirstConsumer   false                  8h
openebs-hostpath            openebs.io/local                                           Delete          WaitForFirstConsumer   false                  8h
openebs-jiva-default        openebs.io/provisioner-iscsi                       

In [128]:
!kubectl get pv

NAME                                       CAPACITY   ACCESS MODES   RECLAIM POLICY   STATUS   CLAIM                          STORAGECLASS     REASON   AGE
pvc-097f3863-4fa9-4303-9195-d84d51a89985   20Gi       RWO            Delete           Bound    kubeflow/mysql-pv-claim        gp2                       54d
pvc-12f27651-af46-4dfb-a181-5c768c87add9   5Gi        RWO            Delete           Bound    kubeflow/ebs-claim             ebs-sc                    6h29m
pvc-36899807-7262-46fe-9627-5b3338abd535   10Gi       RWO            Delete           Bound    kubeflow/katib-mysql           gp2                       54d
pvc-47f293c3-6406-4e1d-ac68-97190bbe8e5f   10Gi       RWO            Delete           Bound    anonymous/workspace-notebook   gp2                       54d
pvc-64da95de-d5fc-4df6-b449-19439dbc7345   20Gi       RWO            Delete           Bound    kubeflow/minio-pv-claim        gp2                       54d
pvc-a252cad0-6cc0-4f08-8c81-d0e257849351   5G         RWO     

In [129]:
!kubectl describe pv pvc-c2fa8c1a-5422-473e-aab9-802c378749c1

Name:            pvc-c2fa8c1a-5422-473e-aab9-802c378749c1
Labels:          <none>
Annotations:     pv.kubernetes.io/provisioned-by: fsx.csi.aws.com
Finalizers:      [kubernetes.io/pv-protection]
StorageClass:    fsx-sc
Status:          Bound
Claim:           kubeflow/fsx-claim
Reclaim Policy:  Delete
Access Modes:    RWX
VolumeMode:      Filesystem
Capacity:        1200Gi
Node Affinity:   <none>
Message:         
Source:
    Type:              CSI (a Container Storage Interface (CSI) volume source)
    Driver:            fsx.csi.aws.com
    VolumeHandle:      fs-0ec1bd32928faa01f
    ReadOnly:          false
    VolumeAttributes:      dnsname=fs-0ec1bd32928faa01f.fsx.us-west-2.amazonaws.com
                           mountname=6uzhvbmv
                           storage.kubernetes.io/csiProvisionerIdentity=1603983866621-8081-fsx.csi.aws.com
Events:                <none>


In [130]:
!kubectl get pvc

NAME                 STATUS   VOLUME                                     CAPACITY   ACCESS MODES   STORAGECLASS     AGE
ebs-claim            Bound    pvc-12f27651-af46-4dfb-a181-5c768c87add9   5Gi        RWO            ebs-sc           6h34m
fsx-claim            Bound    pvc-c2fa8c1a-5422-473e-aab9-802c378749c1   1200Gi     RWX            fsx-sc           115m
katib-mysql          Bound    pvc-36899807-7262-46fe-9627-5b3338abd535   10Gi       RWO            gp2              54d
local-hostpath-pvc   Bound    pvc-a252cad0-6cc0-4f08-8c81-d0e257849351   5G         RWO            local-hostpath   7h59m
metadata-mysql       Bound    pvc-ca36d136-cbfb-4466-9152-2cd389f32fef   10Gi       RWO            gp2              54d
minio-pv-claim       Bound    pvc-64da95de-d5fc-4df6-b449-19439dbc7345   20Gi       RWO            gp2              54d
mysql-pv-claim       Bound    pvc-097f3863-4fa9-4303-9195-d84d51a89985   20Gi       RWO            gp2              54d


In [131]:
!kubectl describe pvc fsx-claim

Name:          fsx-claim
Namespace:     kubeflow
StorageClass:  fsx-sc
Status:        Bound
Volume:        pvc-c2fa8c1a-5422-473e-aab9-802c378749c1
Labels:        <none>
Annotations:   kubectl.kubernetes.io/last-applied-configuration:
                 {"apiVersion":"v1","kind":"PersistentVolumeClaim","metadata":{"annotations":{},"name":"fsx-claim","namespace":"kubeflow"},"spec":{"accessM...
               pv.kubernetes.io/bind-completed: yes
               pv.kubernetes.io/bound-by-controller: yes
               volume.beta.kubernetes.io/storage-provisioner: fsx.csi.aws.com
Finalizers:    [kubernetes.io/pvc-protection]
Capacity:      1200Gi
Access Modes:  RWX
VolumeMode:    Filesystem
Mounted By:    bert-csi-fsx
Events:        <none>


In [195]:
import boto3
fsx = boto3.client('fsx')

In [196]:
response = fsx.describe_file_systems(
    FileSystemIds=[
        'fs-01fd6f8121cbf869e',
    ],
)
print(response)

{'FileSystems': [{'OwnerId': '231218423789', 'CreationTime': datetime.datetime(2020, 10, 29, 19, 33, 56, 351000, tzinfo=tzlocal()), 'FileSystemId': 'fs-01fd6f8121cbf869e', 'FileSystemType': 'LUSTRE', 'Lifecycle': 'CREATING', 'StorageCapacity': 1200, 'StorageType': 'SSD', 'VpcId': 'vpc-05cd6f5a363b289c2', 'SubnetIds': ['subnet-0394df757ad94fdf0'], 'NetworkInterfaceIds': ['eni-09d83e7856a9a89ae', 'eni-06bce9dfb1e6ec548'], 'DNSName': 'fs-01fd6f8121cbf869e.fsx.us-west-2.amazonaws.com', 'KmsKeyId': 'arn:aws:kms:us-west-2:231218423789:key/3b1402d5-3a3d-43ab-9f5e-7ce68d1d7cde', 'ResourceARN': 'arn:aws:fsx:us-west-2:231218423789:file-system/fs-01fd6f8121cbf869e', 'Tags': [{'Key': 'Name', 'Value': 'test'}], 'LustreConfiguration': {'WeeklyMaintenanceStartTime': '5:06:30', 'DataRepositoryConfiguration': {'Lifecycle': 'CREATING', 'ImportPath': 's3://fsx-csi-231218423789', 'ExportPath': 's3://fsx-csi-231218423789/', 'ImportedFileChunkSize': 1024, 'AutoImportPolicy': 'NEW_CHANGED'}, 'DeploymentType'

In [216]:
response = fsx.update_file_system(
    FileSystemId='fs-0814c59615405ca30',
    LustreConfiguration={
        'AutoImportPolicy': 'NEW_CHANGED'
    }
)
print(response)

{'FileSystem': {'OwnerId': '231218423789', 'CreationTime': datetime.datetime(2020, 10, 29, 19, 47, 21, 966000, tzinfo=tzlocal()), 'FileSystemId': 'fs-0814c59615405ca30', 'FileSystemType': 'LUSTRE', 'Lifecycle': 'AVAILABLE', 'StorageCapacity': 1200, 'StorageType': 'SSD', 'VpcId': 'vpc-05cd6f5a363b289c2', 'SubnetIds': ['subnet-0394df757ad94fdf0'], 'NetworkInterfaceIds': ['eni-0af3120d5487ddc63', 'eni-068153de55532aa81'], 'DNSName': 'fs-0814c59615405ca30.fsx.us-west-2.amazonaws.com', 'ResourceARN': 'arn:aws:fsx:us-west-2:231218423789:file-system/fs-0814c59615405ca30', 'Tags': [{'Key': 'CSIVolumeName', 'Value': 'pvc-61624c47-dd5b-4214-9455-c55560294d53'}], 'LustreConfiguration': {'WeeklyMaintenanceStartTime': '4:07:30', 'DataRepositoryConfiguration': {'Lifecycle': 'UPDATING', 'ImportPath': 's3://fsx-csi-231218423789', 'ExportPath': 's3://fsx-csi-231218423789', 'ImportedFileChunkSize': 1024, 'AutoImportPolicy': 'NEW_CHANGED'}, 'DeploymentType': 'SCRATCH_2', 'MountName': 'eigxvbmv', 'CopyTag

In [214]:
!kubectl version

Client Version: version.Info{Major:"1", Minor:"15+", GitVersion:"v1.15.10-eks-bac369", GitCommit:"bac3690554985327ae4d13e42169e8b1c2f37226", GitTreeState:"clean", BuildDate:"2020-02-21T23:37:18Z", GoVersion:"go1.12.12", Compiler:"gc", Platform:"linux/amd64"}
Server Version: version.Info{Major:"1", Minor:"17+", GitVersion:"v1.17.9-eks-4c6976", GitCommit:"4c6976793196d70bc5cd29d56ce5440c9473648e", GitTreeState:"clean", BuildDate:"2020-07-17T18:46:04Z", GoVersion:"go1.13.9", Compiler:"gc", Platform:"linux/amd64"}
